In [6]:
using Suppressor
using MLJ
using Plots
pyplot()

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1186


Plots.PyPlotBackend()

In [10]:
using RDatasets
iris = dataset("datasets", "iris");

┌ Info: Precompiling RDatasets [ce6b1742-4840-55fa-b093-852dadbb1d8b]
└ @ Base loading.jl:1186


In [11]:
X, y = iris[:,1:4], iris[:,5];

### DecisionTreeClassifier example

In [17]:
@load DecisionTreeClassifier

import MLJModels ✔
import DecisionTree 

┌ Info: Precompiling DecisionTree [7806a523-6efd-50cb-b5f6-3fa6f1930dbb]
└ @ Base loading.jl:1186


✔
import MLJModels.DecisionTree_.DecisionTreeClassifier ✔


In [18]:
tree_model = DecisionTreeClassifier(max_depth=4)

DecisionTreeClassifier(pruning_purity = 1.0,
                       max_depth = 4,
                       min_samples_leaf = 1,
                       min_samples_split = 2,
                       min_purity_increase = 0.0,
                       n_subfeatures = 0.0,
                       display_depth = 5,
                       post_prune = false,
                       merge_purity_threshold = 0.9,) @ 7…35

In [19]:
tree_machine = machine(tree_model, X, y)

Machine{DecisionTreeClassifier} @ 9…54


In [20]:
train, test = partition(eachindex(y), 0.7, shuffle=true);

In [21]:
fit!(tree_machine, rows=train)

┌ Info: Training Machine{DecisionTreeClassifier} @ 9…54.
└ @ MLJ /Users/macpro/.julia/packages/MLJ/HXakA/src/machines.jl:130


Machine{DecisionTreeClassifier} @ 9…54


In [22]:
y_hat = predict(tree_machine,X[test,:])

45-element Array{UnivariateFinite{CategoricalString{UInt8},Float64},1}:
 UnivariateFinite{CategoricalString{UInt8},Float64}(
prob_given_level: Dict("virginica"=>0.0,"setosa"=>0.0,"versicolor"=>1.0)
)

 UnivariateFinite{CategoricalString{UInt8},Float64}(
prob_given_level: Dict("virginica"=>0.0,"setosa"=>1.0,"versicolor"=>0.0)
)

 UnivariateFinite{CategoricalString{UInt8},Float64}(
prob_given_level: Dict("virginica"=>0.0,"setosa"=>0.0,"versicolor"=>1.0)
)

 UnivariateFinite{CategoricalString{UInt8},Float64}(
prob_given_level: Dict("virginica"=>0.0,"setosa"=>1.0,"versicolor"=>0.0)
)

 UnivariateFinite{CategoricalString{UInt8},Float64}(
prob_given_level: Dict("virginica"=>0.0,"setosa"=>0.0,"versicolor"=>1.0)
)

 UnivariateFinite{CategoricalString{UInt8},Float64}(
prob_given_level: Dict("virginica"=>0.0,"setosa"=>0.0,"versicolor"=>1.0)
)

 UnivariateFinite{CategoricalString{UInt8},Float64}(
prob_given_level: Dict("virginica"=>1.0,"setosa"=>0.0,"versicolor"=>0.0)
)

 UnivariateFinite{Categor

In [23]:
y_hat[1]

UnivariateFinite{CategoricalString{UInt8},Float64}(
prob_given_level: Dict("virginica"=>0.0,"setosa"=>0.0,"versicolor"=>1.0)
)


Notice that the prediction for an input `X[i,:]` is a 

In [24]:
test_probs = [values(yk.prob_given_level) for yk in y_hat];

In [25]:
typeof(test_probs[1])

Base.ValueIterator{Dict{CategoricalString{UInt8},Float64}}

In [26]:
y_hat[1]

UnivariateFinite{CategoricalString{UInt8},Float64}(
prob_given_level: Dict("virginica"=>0.0,"setosa"=>0.0,"versicolor"=>1.0)
)


In [27]:
y_hat[1].prob_given_level

Dict{CategoricalString{UInt8},Float64} with 3 entries:
  CategoricalString{UInt8} "virginica"  => 0.0
  CategoricalString{UInt8} "setosa"     => 0.0
  CategoricalString{UInt8} "versicolor" => 1.0

In [28]:
d = y_hat[1].prob_given_level

Dict{CategoricalString{UInt8},Float64} with 3 entries:
  CategoricalString{UInt8} "virginica"  => 0.0
  CategoricalString{UInt8} "setosa"     => 0.0
  CategoricalString{UInt8} "versicolor" => 1.0

In [29]:
classnames = [x for x in keys(y_hat[1].prob_given_level)]

3-element Array{CategoricalString{UInt8},1}:
 "virginica" 
 "setosa"    
 "versicolor"

In [30]:
pos_to_class = Dict( pos =>class for (pos,class) in enumerate(classnames))
to_class(x) = pos_to_class[x]

to_class (generic function with 1 method)

In [31]:
most_probable_class(y) = argmax([x for x in values(y.prob_given_level)])

most_probable_class (generic function with 1 method)

In [92]:
# test accuracy
y_hat = predict(tree_machine,X[test,:])
mean(to_class.(most_probable_class.(y_hat)) .== y[test])

0.9555555555555556

In [93]:
using BenchmarkTools

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1186


In [94]:
@btime mean(to_class.(most_probable_class.(y_hat)) .== y[test])

  31.916 μs (173 allocations: 8.48 KiB)


0.9555555555555556

In [95]:
@btime mean(mode.(y_hat) .== y[test])

  25.727 μs (73 allocations: 6.81 KiB)


0.9555555555555556

In [82]:
y_hat = predict(tree_machine,X[test,:])
mean(mode.(y_hat) .== y[test])

0.9555555555555556

In [83]:
# train accuracy
y_hat = predict(tree_machine,X[train,:])
mean(to_class.(most_probable_class.(y_hat)) .== y[train])

1.0

missclassification_rate

In [45]:
@which misclassification_rate

MLJ

In [68]:
y_hat = predict(tree_machine,X[test,:])
misclassification_rate(y_hat, y[test])

0.044444444444444446

In [73]:
mode

mode (generic function with 61 methods)

In [42]:
evaluate!(tree_machine, 
          resampling=Holdout(fraction_train=0.7, shuffle=true),
          measure=misclassification_rate)

┌ Info: Evaluating using a holdout set. 
│ fraction_train=0.7 
│ shuffle=true 
│ measure=MLJ.misclassification_rate 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /Users/macpro/.julia/packages/MLJ/HXakA/src/resampling.jl:91


0.044444444444444446

In [46]:
@which misclassification_rate

MLJ

In [64]:
?misclassification_rate

search: misclassification_rate misclassification_rate2



No documentation found.

`MLJ.misclassification_rate` is a `Function`.

```
# 2 methods for generic function "misclassification_rate":
[1] misclassification_rate(yhat::AbstractArray{#s14,1} where #s14<:Union{CategoricalString, CategoricalValue}, y::AbstractArray{T,1} where T) in MLJ at /Users/macpro/.julia/packages/MLJ/HXakA/src/metrics.jl:94
[2] misclassification_rate(yhat, y::AbstractArray{T,1} where T) in MLJ at /Users/macpro/.julia/packages/MLJ/HXakA/src/metrics.jl:95
```


Notice that fit expects Int64 as type for the outputs

https://github.com/JuliaML/MLDataUtils.jl

In [48]:
function map_class_to_int(y; y_enc_type=Int64)
    class_values = Set(y)
    y_enc = zeros(y_enc_type, length(y))
    label_to_int = Dict(class=>y_enc_type(j) for (j,class) in enumerate(class_values))
    
    for (j,class) in enumerate(class_values)
        y_enc[y .==class] .= y_enc_type(j)
    end
    return y_enc, label_to_int
end

map_class_to_int (generic function with 1 method)

In [49]:
y_enc, label_to_int = map_class_to_int(y; y_enc_type=Int64);

In [50]:
label_to_int

Dict{CategoricalString{UInt8},Int64} with 3 entries:
  CategoricalString{UInt8} "virginica"  => 3
  CategoricalString{UInt8} "versicolor" => 1
  CategoricalString{UInt8} "setosa"     => 2

## Data containers and scientific types

### Data containers and scientific types

The MLJ user should acquaint themselves with some
basic assumptions about the form of data expected by MLJ, as outlined
below. 

```
machine(model::Supervised, X, y) 
machine(model::Unsupervised, X)
```

**Multivariate input.** The input `X` in the above machine
constructors can be any table, where *table* means any data type
supporting the [Tables.jl](https://github.com/JuliaData/Tables.jl)
interface.

> At present our API is more restrictive; see this
> [issue](https://github.com/JuliaData/Tables.jl/issues/74) with
> Tables.jl. If your Tables.jl compatible format is not working in
> MLJ, please post an issue.

In particular, `DataFrame`, `JuliaDB.IndexedTable` and
`TypedTables.Table` objects are supported, as are two Julia native
formats: *column tables* (named tuples of equal length vectors) and
*row tables* (vectors of named tuples sharing the same
keys).

> Certain `JuliaDB.NDSparse` tables can be used for sparse data, but
> this is experimental and undocumented.

**Univariate input.** For models which handle only univariate inputs
(`input_is_multivariate(model)=false`) `X` cannot be a table but is
expected to be some `AbstractVector` type.

**Targets.** The target `y` in the first constructor above must be an
`AbstractVector`. A multivariate target `y` will be a vector of
*tuples*. The tuples need not have uniform length, so some forms of
sequence prediction are supported. Only the element types of `y`
matter (the types of `y[j]` for each `j`). Indeed if a machine accepts
`y` as an argument it will be just as happy with `identity.(y)`.

**Element types.** The types of input and target *elements* has strict
consequences for MLJ's behaviour. 

To articulate MLJ's conventions about data representation, MLJ
distinguishes between *machine* data types on the one hand (`Float64`,
`Bool`, `String`, etc) and *scientific data types* on the other,
represented by new Julia types: `Continuous`, `Count`,
`Multiclass{N}`, `OrderedFactor{N}` and `Unknown`, with obvious
interpretations.  These types are organized in a type
[hierarchy](scitypes.png) rooted in a new abstract type `Found`.

A *scientific type* is any subtype of
`Union{Missing,Found}`. Scientific types have no instances. (They are
used behind the scenes is values for model trait functions.) Such
types appear, for example, when querying model metadata:

```julia
julia> info("DecisionTreeClassifier")[:target_scitype_union]
```

```julia
Finite
```

```julia
subtypes(Finite)
```

```julia
2-element Array{Any,1}:
 Multiclass   
 OrderedFactor
```

This means that the scitype of all elements of `DecisionTreeClassier`
target must be `Multiclass` or `OrderedFactor`.

To see how MLJ will interpret an object `x` appearing in table or
vector input `X`, or target vector `y`, call `scitype(x)`. The fallback
this function is `scitype(::Any) = Unknown`. 

```julia
julia> (scitype(42), scitype(float(π)), scitype("Julia"))
```

```julia
(Count, Continuous, Unknown)
```
    
The table below shows machine types that have scientific types
different from `Unknown`:

`T`                         |     `scitype(x)` for `x::T`
----------------------------|:--------------------------------
`AbstractFloat`             |      `Continuous`
`Integer`                   |        `Count`
`CategoricalValue`          | `Multiclass{N}` where `N = nlevels(x)`, provided `x.pool.ordered == false` 
`CategoricalString`         | `Multiclass{N}` where `N =p nlevels(x)`, provided `x.pool.ordered == false`
`CategoricalValue`          | `OrderedFactor{N}` where `N = nlevels(x)`, provided `x.pool.ordered == true` 
`CategoricalString`         | `OrderedFactor{N}` where `N = nlevels(x)` provided `x.pool.ordered == true`
`Integer`                   | `Count`
`Missing`                   | `Missing`

Here `nlevels(x) = length(levels(x.pool))`.

**Special note on using integers.** According to the above, integers
cannot be used to represent `Multiclass` or `OrderedFactor` data. These can be represented by an unordered or ordered `CategoricalValue`
or `CategoricalString` (automatic if they are elements of a
`CategoricalArray`).

Methods exist to coerce the scientific type of a vector or table (see
below). [Task](working_with_tasks.md) constructors also allow one to
force the data being wrapped to have the desired scientific type.

For more about scientific types and their role, see [Adding Models for
General Use](adding_models_for_general_use.md)





## Other

In [74]:
using MLJ
task = load_iris()


SupervisedTask @ 5…38


In [75]:
using RDatasets
df = dataset("boot", "channing");

In [76]:
(Sex = Multiclass{2},
 Entry = Count,
 Exit = Count,
 Time = Count,
 Cens = Count,)

(Sex = Multiclass{2},
 Entry = Count,
 Exit = Count,
 Time = Count,
 Cens = Count,)

In [77]:
task = supervised(data=df,
                  target=:Exit,
                  ignore=:Time,
                  is_probabilistic=true,
                  types=Dict(:Entry=>Continuous,
                             :Exit=>Continuous,
                             :Cens=>Multiclass))
scitypes(task.X)

UndefVarError: UndefVarError: supervised not defined

## Tasks

- Definition: A task is a set of 3 elements: {data, data interpretator, learning objective}.




We can use tasks to choose learning models.



### `SupervisedTask` type

In [ ]:
supertype(SupervisedTask)

In [ ]:
?SupervisedTask

In [ ]:
methodswith(SupervisedTask)

In [ ]:
task = load_iris()

In [ ]:
?models

In [ ]:
models(task)

In [ ]:
typeof(task)

In [ ]:
X, y = task()

In [ ]:
println(typeof(X))
println(typeof(y))

In [ ]:
X

In [ ]:
@load DecisionTreeClassifier

In [ ]:
tree_model = DecisionTreeClassifier(max_depth=5)

Wrapping the model in data creates a machine which will store training outcomes:

In [ ]:
machine_tree = machine(tree_model, X, y)

In [ ]:
train, test = partition(eachindex(y), 0.7, shuffle=true);

In [ ]:
fit!(tree)

In [ ]:
yhat = predict(tree, X[test,:]);